# Рекомендация тарифов

## Первичное изучение данных

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score 
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())
display(df.describe())
df.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


По предварительному анализу мы можем сделать вывод, что пропущенных данных нет, значения в столбцах `calls` и `messages` можно привести к типу integer

In [3]:
df['calls'] = df['calls'].astype(int)
df['messages'] = df['messages'].astype(int)
df['is_ultra'] = df['is_ultra'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 125.7 KB


Теперь данные готовы к разделению на выборки

In [4]:
df['is_ultra'].value_counts()

0.0    2229
1.0     985
Name: is_ultra, dtype: int64

Как мы можем видеть значений в столбце `is_ultra` равных 1 в несколько раз меньше чем 0, а именно в 2,26 раза. 30,6% значений в столбце равны 1, а 69,4 значений 0.

## Разбитие данных на выборки

In [5]:
features = df.drop('is_ultra',axis=1) #Выделение под признаки все данные кроме значения тарифа
target = df['is_ultra'] #Выделение под цель тариф
features_80, features_valid,  target_80, target_valid = train_test_split(
features, target, test_size=0.2, random_state=45678, stratify = target) #Разделение выборки на валидационную и остальную с отношением 20% к 80%
features_train, features_test, target_train, target_test = train_test_split(
features_80, target_80, test_size=0.25, random_state=45678, stratify = target_80) #Разделение выборки на обучаемую и тестовую с отношением 25% к 75%
target_train_per = (target_train[target_train>0].count())/(target_train.count())
target_train_per

0.3065352697095436

Данные были разделены на 3 разных выборки. Обучаемую - 60%, валидационную - 20%, тестовую - 20%, при этом сохранив изначальное распределение данных

## Исследование моделей

Для обучения модели будут использоваться 3 разные модели, качество которых будет определяться по валидационной выборке

In [6]:
#Переменные для определения лучшей модели
best_model_tree = None
best_model_forest = None
best_model_linear = None
best_accuracy_tree = 0
best_accuracy_forest = 0
best_accuracy_linear = 0
best_depth_tree = 0
best_depth_forest = 0
best_est = 0

In [7]:
for depth in range(1,10): #Определение лучшего результата DecisionTreeClassifier
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train,target_train) #Обучение модели
    prediction_valid = model_tree.predict(features_valid) #Тест модели на валидационной выборке
    accuracy = accuracy_score(target_valid,prediction_valid) #Подсчет доли правильных ответов
    if accuracy>best_accuracy_tree: #Проверка на наилучшее качество
        best_accuracy_tree=accuracy
        best_model_tree=model_tree
        best_depth_tree=depth

In [8]:
for est in range(10, 51): #Определение лучшего результата RandomForestClassifier
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=12345,n_estimators=est,max_depth=depth)
        model_forest.fit(features_train,target_train) #Обучение модели
        prediction_valid = model_forest.predict(features_valid) #Тест модели на валидационной выборке
        accuracy = accuracy_score(target_valid,prediction_valid) #Подсчет доли правильных ответов
        if accuracy > best_accuracy_forest: #Проверка на наилучшее качество
            best_model_forest = model_forest
            best_accuracy_forest = accuracy
            best_est = est
            best_depth_forest = depth

In [9]:
model_logistic = LogisticRegression()  #Создание модели LogisticRegression
model_logistic.fit(features_train,target_train) #Обучение модели
prediction_valid = model_logistic.predict(features_valid) #Тест модели на валидационной выборке
accuracy_logistic = accuracy_score(target_valid,prediction_valid) #Подсчет доли правильных ответов

In [10]:
print('Лучший результат DecisionTreeClassifier:',best_accuracy_tree,' При глубине:', best_depth_tree)
print('Лучший результат RandomForestClassifier:',best_accuracy_forest,' При глубине:', best_depth_forest, 'и количестве лесов:',best_est)
print('Лучший результат LogisticRegression:',accuracy_logistic)
            

Лучший результат DecisionTreeClassifier: 0.7807153965785381  При глубине: 9
Лучший результат RandomForestClassifier: 0.7978227060653188  При глубине: 9 и количестве лесов: 37
Лучший результат LogisticRegression: 0.6967340590979783


Исходя из результатов проверки моделей после обучения на валидационных выборках, можно сделать вывод, что лучше всего себя показала модель RandomForestClassifier при глубине 9 и 37 лесах.

In [11]:
features_common = features_train.merge(features_valid,how='outer') #Объединение признаков двух выборок в одну
target_common = pd.concat([target_train, target_valid]) #Объединение целей двух выборок в одну


model_forest = RandomForestClassifier(random_state=12345,n_estimators=37,max_depth=9)
model_forest.fit(features_common,target_common) #Обучение модели
prediction_valid = model_forest.predict(features_common) #Тест модели на общей выборке
accuracy = accuracy_score(target_common,prediction_valid) #Подсчет доли правильных ответов
accuracy

0.8767016725009724

После обучения модели на общей выборке уровень ее правильных ответов вырос до 87,6%

## Проверка модели на тестовой выборке

In [12]:
prediction_test_forest = best_model_forest.predict(features_test)
accurancy_forest = accuracy_score(target_test,prediction_test_forest)
print('Результата работы RandomForestClassifier:', accurancy_forest)

Результата работы RandomForestClassifier: 0.8180404354587869


По итогам проверки на тестовых выборках можно видеть что модель предсказывают правильный результат с вероятностью около 82% и для выбора подходящего тарифа можно использовать RandomForestClassifier

## Проверка модели на адекватность

In [13]:
dummy = DummyClassifier()
dummy.fit(features_train,target_train)
prediction_dum = dummy.predict(features_valid)
accur = accuracy_score(target_valid,prediction_dum)
accur

0.6936236391912908

Если модель будет выдавать полностью случайные ответы, то так как доля 0 среди всех значений равна около 70%, то модель на случайных ответах будет показывать примерно такой же результат (например модель Dummy)? наши же модели предсказывают правильные ответы с вероятностью в 80% что значит, что они достаточно адекватные